In [2]:
from pydantic import BaseModel, field_serializer
from datetime import datetime

In [4]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer('dt', when_used='always')
    def serialize_dt(self, value):
        print(f"type(value) = {type(value)}")
        return value


In [5]:
m = Model(dt=datetime.now())

In [6]:
m.model_dump()

type(value) = <class 'datetime.datetime'>


{'dt': datetime.datetime(2024, 11, 16, 17, 47, 17, 940470)}

In [7]:
m.model_dump_json()

type(value) = <class 'datetime.datetime'>


'{"dt":"2024-11-16T17:47:17.940470"}'

In [9]:
m = Model(dt=None)
m.model_dump_json()

type(value) = <class 'NoneType'>


'{"dt":null}'

In [10]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer('dt', when_used='unless-none')
    def serialize_dt(self, value):
        print(f"type(value) = {type(value)}")
        return value

In [11]:
m = Model(dt=datetime.now())
m.model_dump()

type(value) = <class 'datetime.datetime'>


{'dt': datetime.datetime(2024, 11, 16, 17, 49, 14, 193968)}

In [12]:
m = Model(dt=None)
m.model_dump_json()

'{"dt":null}'

In [13]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer('dt', when_used='json-unless-none')
    def serialize_dt(self, value):
        print(f"type(value) = {type(value)}")
        return value.strftime('%Y/%-m/%-d %I:%M %p')

In [14]:
m = Model(dt=datetime.now())
m.model_dump_json()

type(value) = <class 'datetime.datetime'>


'{"dt":"2024/11/16 05:52 PM"}'

In [15]:
m.model_dump()

{'dt': datetime.datetime(2024, 11, 16, 17, 52, 41, 357682)}

In [16]:
from pydantic import FieldSerializationInfo

In [17]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer('dt', when_used='unless-none')
    def serialize_dt(self, value, info: FieldSerializationInfo):
        print(f"type(value) = {type(value)}")
        print(f"info = {info}")
        return value.strftime('%Y/%-m/%-d %I:%M %p')

In [18]:
m = Model(dt=datetime.now())
m.model_dump_json()

type(value) = <class 'datetime.datetime'>
info = SerializationInfo(include=None, exclude=None, context=None, mode='json', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False, serialize_as_any=False)


'{"dt":"2024/11/16 05:55 PM"}'

In [19]:
import pytz

In [20]:
def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

In [21]:
def dt_utc_json_serializer(dt: datetime) -> str:
    dt = make_utc(dt)
    return dt.strftime('%Y-%m-%dT%H:%M:%SZ')

In [23]:
dt_utc_json_serializer(datetime.now())

'2024-11-16T18:02:21Z'

In [24]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer('dt', when_used='unless-none')
    def serialize_dt(self, value, info: FieldSerializationInfo):
        if info.mode_is_json():
            return dt_utc_json_serializer(value)
        return make_utc(value)

In [25]:
m = Model(dt=datetime.now())

In [27]:
m.model_dump()

{'dt': datetime.datetime(2024, 11, 16, 18, 3, 37, 344918, tzinfo=<UTC>)}

In [28]:
m.model_dump_json()


'{"dt":"2024-11-16T18:03:37Z"}'

In [29]:
eastern = pytz.timezone('US/Eastern')
dt = eastern.localize(datetime.now())
dt

datetime.datetime(2024, 11, 16, 18, 4, 45, 474481, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [30]:
dt.isoformat()

'2024-11-16T18:04:45.474481-05:00'

In [31]:
m = Model(dt=dt)
m

Model(dt=datetime.datetime(2024, 11, 16, 18, 4, 45, 474481, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>))

In [32]:
m.model_dump()

{'dt': datetime.datetime(2024, 11, 16, 23, 4, 45, 474481, tzinfo=<UTC>)}

In [33]:
m.model_dump_json()

'{"dt":"2024-11-16T23:04:45Z"}'